# Problem 1: VASP Input Files

First, we need to prepare a POSCAR for the diamond cubic structure. To do this, we'll import some classes from the [pymatgen](http://pymatgen.org/) library for python, which is written and maintained by the Materials Project.

In [176]:
from pymatgen import Structure, Lattice, PeriodicSite
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

Now we'll create a lattice for DC silicon.

The diamond cubic structure can be thought of as two interpenetrating face centered cubic (FCC) lattices (Let's start by defining the lattice in the more familiar interpenetrating FCC basis and then simplify it to the primitive unit cell with pymatgen. The coordinates of the atoms in the DC structure are (in terms of the lattice vectors):


$\big[ 0, 0, 0 \big]$, 
$\big[ \frac{1}{4}, \frac{1}{4}, \frac{1}{4} \big]$, 
$\big[ \frac{1}{2}, \frac{1}{2}, 0 \big]$, 
$\big[ \frac{1}{2}, 0, \frac{1}{2}\big]$, 
$\big[ 0, \frac{1}{2}, \frac{1}{2}\big]$,
$\big[ \frac{1}{4}, \frac{3}{4}, \frac{3}{4} \big]$, 
$\big[ \frac{3}{4}, \frac{1}{4}, \frac{3}{4} \big]$, 
$\big[ \frac{3}{4}, \frac{3}{4}, \frac{1}{4} \big]$


In [177]:
# Define the lattice parameter, in Å
#lattice_parameter = # YOUR CODE HERE
lattice_parameter = 5.43

# Create a cubic lattice with the lattice parameter above
cubic_lattice = Lattice.cubic(lattice_parameter)

# Create a list of coordinates (i.e. [[x, y, z], [x2, y2, z2], ...])
#DC_coordinates = # YOUR CODE HERE 
DC_coordinates = [[0, 0, 0], [0.25, 0.25, 0.25], [0.5, 0.5, 0], [0.25, 0.75, 0.75], 
                    [0.5, 0, 0.5], [0.75, 0.25, 0.75], [0, 0.5, 0.5], [0.75, 0.75, 0.25]]

DC_species = ["Si" for coord in DC_coordinates] # creates the list of Si species used when creating the Structure

# Create the structure
Si_DC = Structure(cubic_lattice, species=DC_species, coords=DC_coordinates ) # creates the Structure

# Simplify the structure to a primitive one
Si_DC = SpacegroupAnalyzer(Si_DC).get_primitive_standard_structure()

When you enter "`print(Si_DC)`", you should get something that looks similar to the following (sites may be in different order):

```
Full Formula (Si2)
Reduced Formula: Si
abc   :   3.839590   3.839590   3.839590
angles:  60.000000  60.000000  60.000000
Sites (2)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Si    0     0     0
  1  Si    0.25  0.25  0.25
  ```

In [ ]:
print(Si_DC)

Now, we need to prepare our VASP input files.

First, let's import [Kpoints](http://pymatgen.org/pymatgen.io.vasp.inputs.html#pymatgen.io.vasp.inputs.Kpoints), [Incar](http://pymatgen.org/pymatgen.io.vasp.inputs.html#pymatgen.io.vasp.inputs.Incar), [Potcar](http://pymatgen.org/pymatgen.io.vasp.inputs.html#pymatgen.io.vasp.inputs.Potcar),  and [Pocar](http://pymatgen.org/pymatgen.io.vasp.inputs.html#pymatgen.io.vasp.inputs.Poscar) from *pymatgen* and then write some code to prepare a structure optimization calculation. We'll start off defining where we want everything saved and write a POTCAR corresonding to a PBE calculation of Si there.

In [178]:
from pymatgen.io.vasp.sets import MPRelaxSet
from pymatgen.io.vasp.inputs import Kpoints, Poscar, Potcar, Incar, VaspInput
    
POTCAR = Potcar("Si", functional="PBE")

Now lets use the Poscar class to create a POSCAR file for the DC-Si structure we just wrote.

In [ ]:
POSCAR = Poscar(Si_DC)
POSCAR.write_file("{}/POSCAR".format(directory))

Similarly, we can create a KPOINTS file with the Kpoints class...

In [181]:
# fill in this tuple with desired grid dimensions (e.g. (a, b, c))
grid = "YOUR CODE HERE"
grid = (6, 6, 6)

# We're using the gamma_automatic constructor here to create a grid centered at Gamma.
KPOINTS = Kpoints.monkhorst_automatic(kpts=grid)

print(KPOINTS)

Automatic kpoint scheme
0
Monkhorst
6 6 6



Now let's create an INCAR file. We could do that by editing the example file directly, but let's do it with code instead so we can automate that process later.

In [182]:
incar_params = {"EDIFF" : 0.0004, 
                "EDIFFG" : -0.01, 
                "ENCUT" : "YOUR CODE HERE", 
                "IBRION" : "YOUR CODE HERE", 
                "ISIF" : "YOUR CODE HERE", 
                "ISMEAR" : "YOUR CODE HERE", 
                "SIGMA" : "YOUR CODE HERE" , 
                "NSW" : "YOUR CODE HERE",
                "LREAL" : "Auto"}

incar_params = {"EDIFF" : 0.0004, 
                "EDIFFG" : -0.01, 
                "ENCUT" : 350, 
                "IBRION" : 2, 
                "ISIF" : 3, 
                "ISMEAR" : 0, 
                "SIGMA" : 0.2 , 
                "NSW" : 15,
                "LREAL" : "Auto"}

In [183]:
INCAR = Incar.from_dict(incar_params)
print(INCAR)

EDIFF = 0.0004
EDIFFG = -0.01
ENCUT = 350
IBRION = 2
ISIF = 3
ISMEAR = 0
LREAL = Auto
NSW = 15
SIGMA = 0.2



In [187]:
def write_input_set(POTCAR, POSCAR, INCAR, KPOINTS, directory="new_input_set"):
    
    import os
    if not os.path.exists(directory):
        os.makedirs(directory)

    POTCAR.write_file("{}/POTCAR".format(directory))
    POSCAR.write_file("{}/POSCAR".format(directory))
    INCAR.write_file("{}/INCAR".format(directory))
    KPOINTS.write_file("{}/KPOINTS".format(directory))
    print("Input set written to '{}'".format(directory))

Okay, let's write these to our directory and then run fake_vasp on it!

In [188]:
# Directory where you want to save the files. e.g. "my_part_1/Si_DC"
my_directory = "my_part_1/Si_DC" 

write_input_set(POTCAR, POSCAR, INCAR, KPOINTS, directory=my_directory)

Input set written to 'my_part_1/Si_DC'


In [189]:
!fake_vasp -in "my_part_1/Si_DC"

/bin/sh: fake_vasp: command not found


We can now repeat the same process for the $\beta$-Sn structure. Note we only have to prepare a new POSCAR for this. 

In [195]:
# Define the lattice parameters, in Å
#lattice_parameter_a = # YOUR CODE HERE
#lattice_parameter_c = # YOUR CODE HERE
lattice_parameter_a = 4.9 
lattice_parameter_c = 2.548

BSn_coordinates = [[0, 0, 0], [0.5, 0, 0.25], [0, 0.5, 0.75], [0.5, 0.5, 0.5]]

# Create the tetragonal lattice
tetragonal_lattice = Lattice.tetragonal(lattice_parameter_a, lattice_parameter_c)

# Create a list of coordinates (i.e. [ [x1, y1, z1], [x2, y2, z2], ... ])
#BSn_coordinates = # YOUR CODE HERE 
BSn_coordinates = [[0, 0, 0], [0.5, 0, 0.25], [0, 0.5, 0.75], [0.5, 0.5, 0.5]]

BSn_species = ["Si" for coord in BSn_coordinates] # creates the list of Si species used when creating the Structure

Si_BSn = Structure(tetragonal_lattice, species=BSn_species, coords=BSn_coordinates ) # creates the Structure

In [196]:
my_directory = "my_part_1/Si_BSn" 

POSCAR = Poscar(Si_BSn)

write_input_set(POTCAR, POSCAR, INCAR, KPOINTS, directory=my_directory)

Input set written to 'my_part_1/Si_BSn'
